<a href="https://colab.research.google.com/github/quangminh18112003/DoAnC-/blob/main/XLNNTN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ĐỒ ÁN XỬ LÝ NGÔN NGỮ TỰ NHIÊN
## DỊCH MÁY ANH-PHÁP VỚI MÔ HÌNH ENCODER-DECODER LSTM
---





## Câu 1: Xử lý dữ liệu (Tokenization, Vocabulary, Padding, DataLoader)



### 1.1. Chuẩn bị Môi trường & Dữ liệu

**Mục tiêu:**  
- Clone dataset Multi30K từ GitHub
- Cài đặt các thư viện: SpaCy, NLTK, Matplotlib, PyTorch
- Download mô hình ngôn ngữ cho tiếng Anh và tiếng Pháp

In [ ]:
# 1. Clone dataset (Lấy dữ liệu về)
!git clone https://github.com/multi30k/dataset.git

# 2. Cài đặt thư viện cần thiết
!pip install -U spacy nltk matplotlib
!python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_sm

# 3. Download NLTK data cho BLEU score
import nltk
nltk.download('punkt')

fatal: destination path 'dataset' already exists and is not an empty directory.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 88.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 78.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### 1.2. Load và Tokenization dữ liệu

**Các bước:**
- Đọc file `.gz` (train, val, test)
- Tokenization với SpaCy (`en_core_web_sm` cho tiếng Anh, `fr_core_news_sm` cho tiếng Pháp)
- Đảo ngược chuỗi tiếng Pháp (cải thiện LSTM theo Sutskever et al. 2014)
- Chuẩn bị cho bước xây dựng Vocabulary

In [ ]:
import torch
import torch.nn as nn

# Import thư viện cần thiết cho việc xử lý dữ liệu
print(" Đã import các thư viện cần thiết")

 Đã import các thư viện cần thiết


### 1.3. Xây dựng Từ điển (Vocabulary)

**Mô tả:**  
Chuyển từ vựng sang số (Index) để đưa vào Neural Network.

**Special Tokens:**
- `<PAD>`: Padding
- `<SOS>`: Start of Sentence
- `<EOS>`: End of Sentence
- `<UNK>`: Unknown word

In [ ]:
import spacy
import gzip
import os
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

# 1. Đường dẫn đến dữ liệu
data_dir = 'dataset/data/task1/raw'

# 2. Load mô hình ngôn ngữ để tách từ (Tokenization)
try:
    spacy_fr = spacy.load('fr_core_news_sm')
    spacy_en = spacy.load('en_core_web_sm')
except OSError:
    print("Đang tải thư viện ngôn ngữ Spacy...")
    os.system('python -m spacy download en_core_web_sm')
    os.system('python -m spacy download fr_core_news_sm')
    spacy_fr = spacy.load('fr_core_news_sm')
    spacy_en = spacy.load('en_core_web_sm')

def tokenize_fr(text):
    """Hàm tách từ tiếng Pháp & Đảo ngược chuỗi (cải thiện hiệu năng LSTM)"""
    return [tok.text for tok in spacy_fr.tokenizer(text)][::-1]

def tokenize_en(text):
    """Hàm tách từ tiếng Anh"""
    return [tok.text for tok in spacy_en.tokenizer(text)]

# 3. Hàm đọc file .gz
def load_data(filepath):
    if not os.path.exists(filepath):
        print(f"CẢNH BÁO: Không tìm thấy file {filepath}")
        return []
    with gzip.open(filepath, 'rt', encoding='utf-8') as f:
        return [line.strip() for line in f]

# 4. Load dữ liệu (Train, Val, Test)
print("Đang tải dữ liệu...")

# Train
train_src = load_data(os.path.join(data_dir, 'train.en.gz'))
train_trg = load_data(os.path.join(data_dir, 'train.fr.gz'))

# Validation
val_src = load_data(os.path.join(data_dir, 'val.en.gz'))
val_trg = load_data(os.path.join(data_dir, 'val.fr.gz'))

# Test 2016 Flickr
test_src = load_data(os.path.join(data_dir, 'test_2016_flickr.en.gz'))
test_trg = load_data(os.path.join(data_dir, 'test_2016_flickr.fr.gz'))

# 5. Kiểm tra thông tin dữ liệu
print(f"------------ THỐNG KÊ DỮ LIỆU ------------")
print(f"Số lượng câu Train     : {len(train_src)}")
print(f"Số lượng câu Val       : {len(val_src)}")
print(f"Số lượng câu Test 2016 : {len(test_src)}")
print(f"------------------------------------------")

if len(train_src) > 0:
    print(f"Ví dụ Anh (Train): {train_src[0]}")
    print(f"Ví dụ Pháp (Train): {train_trg[0]}")

Đang tải dữ liệu...
------------ THỐNG KÊ DỮ LIỆU ------------
Số lượng câu Train     : 29000
Số lượng câu Val       : 1014
Số lượng câu Test 2016 : 1000
------------------------------------------
Ví dụ Anh (Train): Two young, White males are outside near many bushes.
Ví dụ Pháp (Train): Deux jeunes hommes blancs sont dehors près de buissons.


### 1.4. Dataset & DataLoader với Padding và Packing

**YÊU CẦU ĐỀ BÀI:**
- Sử dụng `pack_padded_sequence` để xử lý sequences có độ dài khác nhau
- Sắp xếp batch theo độ dài giảm dần (`enforce_sorted=True`)
- Batch size: 32-128

**Lợi ích của pack_padded_sequence:**
- Giảm tính toán không cần thiết trên padding tokens
- Tăng tốc độ training
- Cải thiện hiệu suất mô hình

In [ ]:
from collections import Counter

class Vocabulary:
    def __init__(self, freq_threshold=2, max_size=10000):
        # Khởi tạo 4 token đặc biệt
        self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.stoi = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.freq_threshold = freq_threshold
        self.max_size = max_size

    def __len__(self):
        return len(self.itos)

    def build_vocabulary(self, sentence_list, tokenizer):
        # 1. Đếm tần suất toàn bộ các từ trong dataset
        frequencies = Counter()
        for sentence in sentence_list:
            for word in tokenizer(sentence):
                frequencies[word] += 1

        # 2. Lấy danh sách các từ phổ biến nhất theo max_size
        # Trừ đi 4 vì đã có 4 token đặc biệt (<PAD>, <SOS>, <EOS>, <UNK>)
        most_common_words = frequencies.most_common(self.max_size - 4)

        # 3. Thêm vào từ điển nếu đạt ngưỡng freq_threshold
        idx = 4
        for word, count in most_common_words:
            if count >= self.freq_threshold:
                self.stoi[word] = idx
                self.itos[idx] = word
                idx += 1

    def numericalize(self, text, tokenizer):
        tokenized_text = tokenizer(text)
        return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in tokenized_text
        ]

# --- SỬ DỤNG ---
print("Đang xây dựng từ điển (Giới hạn 10,000 từ)...")

# 🔧 CHỐNG OVERFITTING: freq_threshold=2 loại bỏ từ xuất hiện chỉ 1 lần
vocab_en = Vocabulary(freq_threshold=2, max_size=10000)
vocab_en.build_vocabulary(train_src, tokenize_en)

vocab_fr = Vocabulary(freq_threshold=2, max_size=10000)
vocab_fr.build_vocabulary(train_trg, tokenize_fr)

print(f"✅ Kích thước từ điển EN: {len(vocab_en):,} từ")
print(f"✅ Kích thước từ điển FR: {len(vocab_fr):,} từ")
print(f"🔧 freq_threshold=2 (loại bỏ từ xuất hiện chỉ 1 lần)")
# Lưu ý: Nếu dataset nhỏ hơn 10,000 từ vựng thì nó sẽ lấy hết số từ hiện có.

Đang xây dựng từ điển (Giới hạn 10,000 từ)...
✅ Kích thước từ điển EN: 6,191 từ
✅ Kích thước từ điển FR: 6,555 từ
🔧 freq_threshold=2 (loại bỏ từ xuất hiện chỉ 1 lần)


---

## Câu 2: Triển khai mô hình Encoder-Decoder LSTM với context vector có định hướng



### 2.1. Xây dựng Encoder-Decoder LSTM với Context Vector

**Kiến trúc Seq2Seq (theo yêu cầu đề bài):**

**Encoder:**
- Input: Chuỗi token tiếng Anh → embedding (size 256)
- LSTM (2 layers, hidden=512, dropout=0.5)
- Sử dụng `pack_padded_sequence` để bỏ qua padding
- Output: Context Vector = (hidden state, cell state) cuối cùng

**Decoder:**
- Input: Context Vector từ Encoder + Target sequence
- LSTM (2 layers, hidden=512, dropout=0.5)
- Linear layer → Vocabulary size
- Teacher Forcing với Scheduled Sampling (0.5)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_lengths):
        # src = [src len, batch size]
        # src_lengths = [batch size] - độ dài thực của mỗi câu
        embedded = self.dropout(self.embedding(src))

        # Pack padded sequence để LSTM bỏ qua padding (yêu cầu đề bài)
        packed_embedded = nn.utils.rnn.pack_padded_sequence(
            embedded, src_lengths.cpu(), enforce_sorted=True
        )

        packed_outputs, (hidden, cell) = self.rnn(packed_embedded)

        # Unpack sequence (không dùng outputs nên có thể bỏ qua)
        # outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs)

        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, src_lengths, teacher_forcing_ratio=0.5):
        trg_len = trg.shape[0]
        batch_size = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        hidden, cell = self.encoder(src, src_lengths)

        input = trg[0,:] # Token <SOS>

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1

        return outputs

print(" Đã định nghĩa kiến trúc Encoder-Decoder LSTM")

 Đã định nghĩa kiến trúc Encoder-Decoder LSTM


### 2.2. Tạo DataLoader & Khởi tạo mô hình

**Dataset & DataLoader:**
- Sử dụng class `Multi30kDataset` tùy chỉnh
- Padding sequences với `pad_sequence`
- Sắp xếp theo độ dài giảm dần (cho `pack_padded_sequence`)
- Batch size: 32 (giảm để chống overfitting)

**Hyperparameters (theo đề bài):**
- Embedding dimension: 256
- Hidden dimension: 512
- Number of layers: 2
- Dropout: 0.5 (mức cao nhất để chống overfitting)
- Device: GPU nếu có, CPU nếu không

In [ ]:
class Multi30kDataset(Dataset):
    def __init__(self, src_lines, trg_lines, vocab_src, vocab_trg):
        self.src_lines = src_lines
        self.trg_lines = trg_lines
        self.vocab_src = vocab_src
        self.vocab_trg = vocab_trg

    def __len__(self):
        return len(self.src_lines)

    def __getitem__(self, index):
        src_text = self.src_lines[index]
        trg_text = self.trg_lines[index]

        src_indices = [1] + self.vocab_src.numericalize(src_text, tokenize_en) + [2] # Thêm SOS, EOS
        trg_indices = [1] + self.vocab_trg.numericalize(trg_text, tokenize_fr) + [2]

        return torch.tensor(src_indices), torch.tensor(trg_indices)

def collate_fn(batch):
    """Hàm này để padding các câu cho bằng độ dài nhau trong 1 batch"""
    src_batch, trg_batch = [], []
    src_lengths = []

    for src_item, trg_item in batch:
        src_batch.append(src_item)
        trg_batch.append(trg_item)
        src_lengths.append(len(src_item))

    # Sắp xếp theo độ dài giảm dần (yêu cầu của pack_padded_sequence)
    src_lengths = torch.tensor(src_lengths)
    sorted_indices = src_lengths.argsort(descending=True)

    src_batch = [src_batch[i] for i in sorted_indices]
    trg_batch = [trg_batch[i] for i in sorted_indices]
    src_lengths = src_lengths[sorted_indices]

    src_batch = pad_sequence(src_batch, padding_value=0) # 0 là <PAD>
    trg_batch = pad_sequence(trg_batch, padding_value=0)

    return src_batch, trg_batch, src_lengths

# 🔧 CHỐNG OVERFITTING: Tăng batch size để giảm noise
BATCH_SIZE = 128

# Tạo DataLoader cho Train
train_dataset = Multi30kDataset(train_src, train_trg, vocab_en, vocab_fr)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

# Tạo DataLoader cho Validation
val_dataset = Multi30kDataset(val_src, val_trg, vocab_en, vocab_fr)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# Tạo DataLoader cho Test
test_dataset = Multi30kDataset(test_src, test_trg, vocab_en, vocab_fr)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

print(f"✅ Đã tạo DataLoader với batch_size={BATCH_SIZE}")

# ============================================================================
# KHỞI TẠO MÔ HÌNH - GIẢM MODEL COMPLEXITY ĐỂ CHỐNG OVERFITTING
# ============================================================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
INPUT_DIM = len(vocab_en)
OUTPUT_DIM = len(vocab_fr)

# 🔧 GIẢM MODEL SIZE để chống overfitting
ENC_EMB_DIM = 512      # Giữ 256
DEC_EMB_DIM = 512      # Giữ 256
HID_DIM = 512          # 🔧 GIẢM từ 512 → 256 (giảm complexity)
N_LAYERS = 2           # Giữ 2 layers
ENC_DROPOUT = 0.5      # Giữ 0.5
DEC_DROPOUT = 0.5      # Giữ 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
model = Seq2Seq(enc, dec, device).to(device)

print(f"✅ Mô hình đã được khởi tạo trên: {device}")
print(f"📊 Cấu hình: EMB={ENC_EMB_DIM}, HID={HID_DIM} (GIẢM), LAYERS={N_LAYERS}, DROPOUT={ENC_DROPOUT}")
print(f"✅ Số tham số: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

✅ Đã tạo DataLoader với batch_size=128
✅ Mô hình đã được khởi tạo trên: cuda
📊 Cấu hình: EMB=512, HID=512 (GIẢM), LAYERS=2, DROPOUT=0.5
✅ Số tham số: 18,293,659


## Câu 3: Huấn luyện, Đánh giá, Early Stopping, Lưu Checkpoint

### 3.1. Training Loop với Early Stopping

**Thiết lập (theo yêu cầu đề bài):**
- Optimizer: Adam (lr=0.001, weight_decay=1e-5)
- Loss: CrossEntropyLoss (ignore padding, label_smoothing=0.1)
- Epochs: 10-20 (max 20)
- Early Stopping: patience = 3
- Gradient Clipping: 1.0
- Checkpoint: Lưu `best_model.pth` khi val_loss giảm

**Kỹ thuật chống Overfitting:**
- Dropout 0.5
- L2 Regularization (weight_decay)
- Label Smoothing 0.1
- Scheduled Sampling (TF: 1.0 → 0.5)
- Early Stopping

In [ ]:
import torch.optim as optim

# 🔧 CHỐNG OVERFITTING MẠNH: Tăng weight_decay (L2 regularization)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# 🔧 LABEL SMOOTHING MẠNH HƠN: 0.1 → 0.2
criterion = nn.CrossEntropyLoss(ignore_index=0, label_smoothing=0.2)

# 🔧 THÊM LEARNING RATE SCHEDULER để giảm lr khi val_loss không giảm
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

def train(model, iterator, optimizer, criterion, clip, epoch, total_epochs):
    """
    🔧 ĐÃ THÊM SCHEDULED SAMPLING + WORD DROPOUT:
    - Teacher forcing ratio giảm dần từ 1.0 → 0.5 theo epoch
    - Word dropout: Ngẫu nhiên thay một số từ input bằng <UNK>
    """
    model.train()
    epoch_loss = 0

    # 🔧 SCHEDULED SAMPLING: Giảm dần teacher forcing
    teacher_forcing_ratio = max(0.5, 1.0 - 0.5 * (epoch / total_epochs))

    for i, (src, trg, src_lengths) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)

        # 🔧 WORD DROPOUT: Ngẫu nhiên thay 10% từ trong src bằng <UNK> (index 3)
        if model.training:
            mask = torch.rand(src.shape) < 0.1
            mask = mask.to(device)
            # Không dropout padding (index 0) và special tokens
            mask = mask & (src > 3)
            src = src.masked_fill(mask, 3)  # 3 = <UNK>

        optimizer.zero_grad()
        output = model(src, trg, src_lengths, teacher_forcing_ratio)

        # Reshape để tính loss
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator), teacher_forcing_ratio

def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, (src, trg, src_lengths) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)
            output = model(src, trg, src_lengths, teacher_forcing_ratio=0) # Không dùng teacher forcing khi eval

            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)

# ============================================================================
# HUẤN LUYỆN VỚI EARLY STOPPING + LR SCHEDULER
# ============================================================================
CLIP = 1.0              # Gradient clipping
N_EPOCHS = 20           # Tối đa 20 epochs
PATIENCE = 3            # Early stopping patience = 3

best_val_loss = float('inf')
epochs_no_improve = 0

# Lưu lịch sử để vẽ biểu đồ
train_losses = []
val_losses = []

for epoch in range(N_EPOCHS):
    # Train với scheduled sampling + word dropout
    train_loss, tf_ratio = train(model, train_loader, optimizer, criterion, CLIP, epoch, N_EPOCHS)

    # Validation
    val_loss = evaluate(model, val_loader, criterion)

    # 🔧 Learning Rate Scheduler
    old_lr = optimizer.param_groups[0]['lr']
    scheduler.step(val_loss)
    new_lr = optimizer.param_groups[0]['lr']

    # Lưu lịch sử
    train_losses.append(train_loss)
    val_losses.append(val_loss)

    # Tính gap
    gap = abs(val_loss - train_loss)

    # In kết quả
    lr_info = f" | LR: {new_lr:.6f}" if new_lr != old_lr else ""
    print(f'Epoch: {epoch+1:02}/{N_EPOCHS} | Train: {train_loss:.3f} | Val: {val_loss:.3f} | Gap: {gap:.3f} | TF: {tf_ratio:.2f}{lr_info}')

    # Early Stopping Logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), 'best_model.pth')
        print(f'   --> ✅ Lưu checkpoint mới (Val Loss: {val_loss:.3f})')
    else:
        epochs_no_improve += 1
        print(f'   --> ⚠️ Val Loss không cải thiện ({epochs_no_improve}/{PATIENCE})')

        if epochs_no_improve >= PATIENCE:
            print(f'\n🛑 Early Stopping: Val Loss không giảm sau {PATIENCE} epoch.')
            print(f'✅ Kết thúc huấn luyện tại Epoch {epoch+1}')
            break

print("=" * 60)
print(f"HOÀN TẤT! Best Validation Loss: {best_val_loss:.3f}")
if len(train_losses) > 0 and len(val_losses) > 0:
    final_gap = abs(val_losses[-1] - train_losses[-1])
    print(f"Train/Val Gap cuối cùng: {final_gap:.3f}")
    if final_gap < 0.5:
        print("✅ Mô hình tổng quát hóa RẤT TỐT (gap < 0.5)")
    elif final_gap < 1.0:
        print("✅ Mô hình tổng quát hóa TỐT (gap < 1.0)")
    elif final_gap < 1.5:
        print("⚠️ Overfitting nhẹ (1.0 < gap < 1.5)")
    else:
        print("❌ Overfitting nặng (gap >= 1.5)")
print("=" * 60)

# Load lại mô hình tốt nhất
model.load_state_dict(torch.load('best_model.pth'))
print("✅ Đã tải lại mô hình tốt nhất từ checkpoint 'best_model.pth'")

Epoch: 01/20 | Train: 5.669 | Val: 6.675 | Gap: 1.006 | TF: 1.00
   --> ✅ Lưu checkpoint mới (Val Loss: 6.675)
Epoch: 02/20 | Train: 4.887 | Val: 6.582 | Gap: 1.695 | TF: 0.97
   --> ✅ Lưu checkpoint mới (Val Loss: 6.582)
Epoch: 03/20 | Train: 4.641 | Val: 6.372 | Gap: 1.731 | TF: 0.95
   --> ✅ Lưu checkpoint mới (Val Loss: 6.372)
Epoch: 04/20 | Train: 4.517 | Val: 6.101 | Gap: 1.584 | TF: 0.93
   --> ✅ Lưu checkpoint mới (Val Loss: 6.101)
Epoch: 05/20 | Train: 4.426 | Val: 5.919 | Gap: 1.492 | TF: 0.90
   --> ✅ Lưu checkpoint mới (Val Loss: 5.919)
Epoch: 06/20 | Train: 4.358 | Val: 5.854 | Gap: 1.496 | TF: 0.88
   --> ✅ Lưu checkpoint mới (Val Loss: 5.854)
Epoch: 07/20 | Train: 4.313 | Val: 5.774 | Gap: 1.461 | TF: 0.85
   --> ✅ Lưu checkpoint mới (Val Loss: 5.774)
Epoch: 08/20 | Train: 4.284 | Val: 5.604 | Gap: 1.320 | TF: 0.82
   --> ✅ Lưu checkpoint mới (Val Loss: 5.604)
Epoch: 09/20 | Train: 4.248 | Val: 5.606 | Gap: 1.358 | TF: 0.80
   --> ⚠️ Val Loss không cải thiện (1/3)


## Câu 4: Hàm translate() hoạt động với câu mới

**Yêu cầu bắt buộc của đề bài:**  
Viết hàm `translate(sentence: str) -> str` nhận câu tiếng Anh, trả về câu tiếng Pháp.

**Pipeline:**
1. Tokenize câu tiếng Anh với SpaCy
2. Chuyển tokens thành indices (numericalize)
3. Encoder → Context Vector (hidden, cell)
4. Decoder với Greedy Decoding → từng token
5. Đảo ngược kết quả (vì đã reverse khi tokenize FR)
6. Dừng khi gặp `<EOS>` hoặc đạt max_len=50
7. Trả về string tiếng Pháp

In [ ]:
def translate_sentence(sentence, src_tokenizer, vocab_src, vocab_trg, model, device, max_len=50):
    """
    Dịch một câu từ tiếng Anh sang tiếng Pháp

    Args:
        sentence: Câu tiếng Anh (string hoặc list tokens)
        src_tokenizer: Hàm tokenize cho tiếng Anh
        vocab_src: Từ điển tiếng Anh
        vocab_trg: Từ điển tiếng Pháp
        model: Mô hình Seq2Seq
        device: CPU hoặc GPU
        max_len: Độ dài tối đa câu dịch (mặc định 50 theo đề bài)

    Returns:
        List tokens tiếng Pháp (đã đảo ngược về đúng thứ tự)
    """
    model.eval()

    # Xử lý câu đầu vào
    if isinstance(sentence, str):
        tokens = [token.text.lower() for token in src_tokenizer(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # Thêm token đặc biệt
    tokens = ['<SOS>'] + tokens + ['<EOS>']

    # Chuyển thành tensor
    src_indexes = [vocab_src.stoi.get(token, vocab_src.stoi['<UNK>']) for token in tokens]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    # Tạo src_lengths cho encoder (độ dài thực của câu)
    src_length = torch.LongTensor([len(src_indexes)]).to(device)

    # Encoder
    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor, src_length)

    # Decoder (Greedy Decoding)
    trg_indexes = [vocab_trg.stoi['<SOS>']]

    for i in range(max_len):
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)

        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token)

        # Dừng khi gặp <EOS>
        if pred_token == vocab_trg.stoi['<EOS>']:
            break

    # Chuyển index về tokens
    trg_tokens = [vocab_trg.itos[i] for i in trg_indexes]

    # Bỏ <SOS> và <EOS>
    trg_tokens = trg_tokens[1:]
    if trg_tokens and trg_tokens[-1] == '<EOS>':
        trg_tokens = trg_tokens[:-1]

    # ĐẢO NGƯỢC vì đã reverse khi train
    trg_tokens = trg_tokens[::-1]

    return trg_tokens

# ============================================================================
# HÀM TRANSLATE() - YÊU CẦU BẮT BUỘC CỦA ĐỀ BÀI
# ============================================================================
def translate(sentence: str) -> str:
    """
    Dịch câu tiếng Anh sang tiếng Pháp (YÊU CẦU ĐỀ BÀI)

    Args:
        sentence: Câu tiếng Anh (string)

    Returns:
        Câu tiếng Pháp đã dịch (string)
    """
    tokens = translate_sentence(sentence, spacy_en, vocab_en, vocab_fr, model, device)
    return " ".join(tokens)

# ============================================================================
# DEMO: Dịch thử vài câu
# ============================================================================
print("\n" + "=" * 60)
print("DEMO: DỊCH MÁY ANH - PHÁP")
print("=" * 60)

test_sentences = [
    "A dog is running in the garden.",
    "Two young girls are playing with toys.",
    "A man is riding a bicycle.",
    "The children are eating ice cream.",
    "A woman is reading a book."
]

for sentence in test_sentences:
    # Sử dụng hàm translate() theo yêu cầu đề bài
    translation_text = translate(sentence)

    print(f"\n🇬🇧 Tiếng Anh: {sentence}")
    print(f"🇫🇷 Tiếng Pháp: {translation_text}")
    print("-" * 60)

## Câu 5: BLEU Score + Biểu đồ Loss + Phân tích lỗi




### 5.1. Đánh giá BLEU Score

**BLEU (Bilingual Evaluation Understudy):**
- So sánh n-grams giữa bản dịch và ground truth
- Sử dụng `nltk.translate.bleu_score.sentence_bleu`
- Tính BLEU cho từng câu, sau đó lấy trung bình
- Đánh giá trên test set (test_2016_flickr)

**Smoothing Function:**
- Sử dụng `SmoothingFunction().method4`
- Tránh BLEU = 0 khi thiếu n-gram matches
- Cho kết quả ổn định hơn với câu ngắn

**Cách tính:**
1. Với mỗi câu: `sentence_bleu([reference_tokens], hypothesis_tokens, smoothing_function=smooth)`
2. Lấy trung bình BLEU của tất cả câu trong test set

**Test Loss:**
- Đánh giá trên test set
- Kiểm tra overfitting: So sánh với train/val loss

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Hàm tính BLEU
def calculate_bleu(data_src, data_trg, src_tokenizer, vocab_src, vocab_trg, model, device):
    """
    Tính BLEU score trung bình trên tập test bằng sentence_bleu
    (theo yêu cầu đề bài: nltk.translate.bleu_score.sentence_bleu)

    Sử dụng smoothing function để tránh BLEU = 0 khi thiếu n-gram
    """
    bleu_scores = []
    model.eval()

    # Khởi tạo smoothing function (method4 - Chen and Cherry 2014)
    smooth = SmoothingFunction().method4

    total = len(data_src)
    print(f"Đang tính toán BLEU score trên {total} câu test...")
    print("Sử dụng smoothing function (method4) để xử lý n-gram không match")
    print("Vui lòng đợi, quá trình này có thể mất 3-5 phút.")

    # Tính trên TOÀN BỘ tập test (theo yêu cầu đề bài)
    for idx, (src, trg) in enumerate(zip(data_src, data_trg)):
        # In progress mỗi 100 câu
        if (idx + 1) % 100 == 0:
            print(f"  → Đã xử lý {idx + 1}/{total} câu ({(idx+1)/total*100:.1f}%)")

        # QUAN TRỌNG: Tokenize bình thường (KHÔNG reverse) để so sánh
        # vì translate_sentence đã reverse output về thứ tự đúng rồi
        trg_tokens = [tok.text for tok in spacy_fr.tokenizer(trg)]

        pred_tokens = translate_sentence(src, src_tokenizer, vocab_src, vocab_trg, model, device)
        if pred_tokens and pred_tokens[-1] == '<EOS>':
            pred_tokens = pred_tokens[:-1]

        # Tính BLEU cho từng câu bằng sentence_bleu với smoothing
        # references phải là list of lists (có thể có nhiều bản dịch tham khảo)
        bleu = sentence_bleu([trg_tokens], pred_tokens, smoothing_function=smooth)
        bleu_scores.append(bleu)

    print(f"   Hoàn tất {total}/{total} câu (100%)\n")

    # Trả về BLEU trung bình
    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    return avg_bleu

# ==============================================================================
# THỰC THI - Đánh giá trên Test Set
# ==============================================================================
print("\n" + "=" * 60)
print("ĐÁNH GIÁ MÔ HÌNH TRÊN TẬP TEST")
print("=" * 60)

# Tính Test Loss
test_loss = evaluate(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.3f}')

# Tính BLEU Score
bleu_score = calculate_bleu(test_src, test_trg, spacy_en, vocab_en, vocab_fr, model, device)
print(f'BLEU Score: {bleu_score*100:.2f}')
print("=" * 60)

### 5.2. Vẽ Biểu đồ Train/Val Loss

**Mục đích:** Phát hiện Overfitting/Underfitting

**Phân tích:**
- Train Loss ↓, Val Loss ↑ → Overfitting
- Cả hai ↓ → Học tốt
- Cả hai cao → Underfitting

In [ ]:
import matplotlib.pyplot as plt

# Vẽ biểu đồ so sánh Train Loss vs Val Loss
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_losses)+1), train_losses, 'b-', label='Train Loss', marker='o')
plt.plot(range(1, len(val_losses)+1), val_losses, 'r-', label='Validation Loss', marker='s')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Train Loss vs Validation Loss', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('training_history.png', dpi=300)
plt.show()

print(" Đã lưu biểu đồ vào file 'training_history.png'")

### 5.3. Demo dịch và phân tích lỗi

**Demo 5 câu test:**
- Hiển thị câu gốc (EN)
- Hiển thị ground truth (FR)
- Hiển thị prediction (FR)
- So sánh và phân tích

In [ ]:
# --- DEMO KẾT QUẢ ---
print("-" * 50)
print("KẾT QUẢ DỊCH THỬ NGHIỆM:")
print("-" * 50)

# Lấy ngẫu nhiên vài câu trong tập test để dịch
indices = [0, 10, 50, 100, 150]

for i in indices:
    src = test_src[i]
    trg = test_trg[i]

    print(f'Câu gốc (EN): {src}')
    print(f'Câu mẫu (FR): {trg}')

    # Dịch câu
    translation = translate_sentence(src, spacy_en, vocab_en, vocab_fr, model, device)
    translation_text = " ".join(translation)

    print(f'Máy dịch (Pred): {translation_text}')
    print("-" * 50)

### 5.4. Phân tích các vấn đề đã xử lý

**1. Độ dài chuỗi khác nhau:**
- Giải pháp: pad_sequence + collate_fn

**2. Teacher Forcing & Exposure Bias:**
- Giải pháp: teacher_forcing_ratio = 0.5

**3. Overfitting:**
- Giải pháp: Dropout + Early Stopping

**4. Loss không giảm:**
- Giải pháp: Gradient clipping + ignore padding

In [ ]:
# ============================================================================
# PHÂN TÍCH CÁC VẤN ĐỀ ĐÃ XỬ LÝ
# ============================================================================

print("\n" + "=" * 60)
print("PHÂN TÍCH CÁC VẤN ĐỀ ĐÃ XỬ LÝ")
print("=" * 60)

# Phân tích độ dài câu
print("\n1. PHÂN TÍCH ĐỘ DÀI CÂU:")
print("-" * 40)

import numpy as np

train_src_lengths = [len(tokenize_en(s)) for s in train_src[:1000]]
train_trg_lengths = [len(tokenize_fr(s)) for s in train_trg[:1000]]

print(f"Độ dài trung bình (EN): {np.mean(train_src_lengths):.1f} từ")
print(f"Độ dài trung bình (FR): {np.mean(train_trg_lengths):.1f} từ")
print(f"Độ dài max (EN): {np.max(train_src_lengths)} từ")
print(f"Độ dài max (FR): {np.max(train_trg_lengths)} từ")

# Teacher Forcing
print("\n2. TEACHER FORCING:")
print("-" * 40)
print(f"✅ Đã áp dụng Scheduled Sampling:")
print(f"   → Bắt đầu: teacher_forcing_ratio = 1.0")
print(f"   → Kết thúc: teacher_forcing_ratio = 0.5")
print(f"   → Giảm dần theo epoch để giảm exposure bias")

# Kiểm tra Overfitting
print("\n3. PHÒNG CHỐNG OVERFITTING:")
print("-" * 40)
print(f"✅ Dropout: {ENC_DROPOUT}")
print(f"✅ Early Stopping: patience = {PATIENCE}")
print(f"✅ Label Smoothing: 0.1")
print(f"✅ Weight Decay: 1e-5")
print(f"✅ Batch Size: {BATCH_SIZE}")
print(f"📊 Best val loss: {best_val_loss:.3f}")

if len(train_losses) > 3:
    last_train = train_losses[-1]
    last_val = val_losses[-1]
    gap = abs(last_val - last_train)

    if gap < 1.0:
        print(f"✅ Mô hình tổng quát hóa tốt (gap = {gap:.3f})")
    else:
        print(f"⚠️ Có dấu hiệu overfitting (gap = {gap:.3f})")

# Loss function
print("\n4. PHÂN TÍCH LOSS:")
print("-" * 40)
print(f"Loss function: CrossEntropyLoss")
print(f"✅ ignore_index=0 (loại bỏ padding token)")
print(f"✅ label_smoothing=0.1 (giảm overconfidence)")

print("\n" + "=" * 60)

## Câu 6: Phân tích 5 ví dụ dịch + Đề xuất cải tiến



### 6.1. Phân tích lỗi phổ biến

**5 câu phân tích theo độ khó:**
- 2 câu **DỄ**: Câu ngắn, từ phổ biến (VD: "A dog is running.")
- 2 câu **TRUNG BÌNH**: Câu dài hơn, cấu trúc phức tạp (VD: "A woman is reading a book in the park.")
- 1 câu **KHÓ**: Câu dài, có thể có từ hiếm

**Lỗi OOV (Out of Vocabulary):**
- Từ hiếm → `<UNK>`
- Đề xuất: BPE tokenization hoặc giảm freq_threshold

**Lỗi cấu trúc câu:**
- Câu dài bị quên thông tin đầu (do context vector cố định)
- Đề xuất: Attention Mechanism

**Lỗi ngữ pháp:**
- Chưa học đủ patterns
- Đề xuất: Beam Search, more data

In [ ]:
# ============================================================================
# PHÂN TÍCH LỖI VÀ ĐỀ XUẤT CẢI TIẾN
# ============================================================================

print("\n" + "=" * 60)
print("PHÂN TÍCH LỖI VÀ ĐỀ XUẤT CẢI TIẾN")
print("=" * 60)

# 6.1. Phân tích một số câu dịch (Dễ → Trung bình → Khó)
print("\n6.1. PHÂN TÍCH CÂU DỊCH THEO ĐỘ KHÓ:")
print("-" * 50)

# 5 câu phân tích: 2 dễ + 2 trung bình + 1 khó
analysis_samples = [
    # 2 câu DỄ (câu ngắn, từ phổ biến)
    ("A dog is running.", "Dễ"),
    ("The man is eating.", "Dễ"),
    # 2 câu TRUNG BÌNH (câu dài hơn, cấu trúc phức tạp hơn)
    ("A woman is reading a book in the park.", "Trung bình"),
    ("Two children are playing with a ball.", "Trung bình"),
    # 1 câu KHÓ (câu dài, có thể có từ hiếm)
    ("The professional photographer is capturing beautiful moments during sunset.", "Khó"),
]

for idx, (en_sentence, difficulty) in enumerate(analysis_samples, 1):
    pred = translate_sentence(en_sentence, spacy_en, vocab_en, vocab_fr, model, device)
    pred_text = " ".join(pred)

    print(f"\n📌 Câu {idx} [{difficulty}]")
    print(f"🇬🇧 EN: {en_sentence}")
    print(f"🇫🇷 FR: {pred_text}")

    # Đếm số từ <UNK>
    unk_count = sum(1 for token in tokenize_en(en_sentence) if token not in vocab_en.stoi)
    if unk_count > 0:
        print(f"   ⚠️ Phát hiện {unk_count} từ OOV (không có trong từ điển)")
    else:
        print(f"   ✅ Tất cả từ đều có trong từ điển")

print("\n" + "-" * 50)
print("📊 NHẬN XÉT:")
print("   • Câu DỄ: Thường dịch chính xác vì từ phổ biến")
print("   • Câu TRUNG BÌNH: Có thể sai ngữ pháp nhỏ")
print("   • Câu KHÓ: Dễ có <UNK> và mất thông tin ngữ cảnh")

# 6.2. Thống kê từ vựng
print("\n\n6.2. THỐNG KÊ TỪ VỰNG:")
print("-" * 40)
print(f"📖 Kích thước vocab EN: {len(vocab_en):,} từ")
print(f"📖 Kích thước vocab FR: {len(vocab_fr):,} từ")
print(f"🔧 Frequency threshold: {vocab_en.freq_threshold}")
print(f"\n💡 Nếu vocab nhỏ → nhiều <UNK> → dịch kém chính xác")
print(f"   Giải pháp: Giảm freq_threshold hoặc dùng BPE tokenization")

print("\n" + "=" * 60)

### 6.2. Đề xuất cải tiến

**Các kỹ thuật cải tiến kiến trúc:**

1. **Attention Mechanism (Cơ chế Tập trung)**
   - Giúp mô hình "nhìn" vào từng phần quan trọng của câu nguồn khi decode
   - Thay vì dùng 1 context vector cố định, decoder có thể truy cập toàn bộ hidden states của encoder
   
2. **Beam Search (Tìm kiếm Chùm)**
   - Tìm kiếm nhiều khả năng (k candidates) thay vì chỉ 1 (greedy)
   - Giúp tìm được bản dịch tổng thể tốt hơn bằng cách xem xét nhiều đường đi
   
3. **Subword Tokenization - BPE (Byte Pair Encoding)**
   - Chia từ thành các đơn vị nhỏ hơn (ví dụ: "playing" → "play" + "ing")
   - Xử lý từ hiếm và từ ngoài từ điển (OOV) hiệu quả hơn
   
4. **Transformer**
   - Kiến trúc hiện đại thay thế LSTM
   - Sử dụng self-attention, xử lý song song, nhanh và chính xác hơn

**Kỹ thuật cải tiến huấn luyện:**
1. ReduceLROnPlateau - Giảm learning rate khi loss không giảm
2. Scheduled Sampling - Giảm dần teacher forcing theo thời gian
3. Data Augmentation - Tăng dữ liệu bằng back-translation

In [ ]:
# 6.2. Đề xuất cải tiến
print("\n" + "=" * 60)
print("ĐỀ XUẤT CẢI TIẾN")
print("=" * 60)

print("\n CÁC KỸ THUẬT CẢI TIẾN KIẾN TRÚC:")
print("=" * 60)

architecture_improvements = [
    ("Attention Mechanism",
     "Giúp decoder 'chú ý' vào từng phần câu nguồn thay vì dùng 1 context vector cố định",
     "Tăng BLEU 5-10%, đặc biệt tốt với câu dài"),

    ("Beam Search",
     "Giữ lại k khả năng tốt nhất mỗi bước thay vì chỉ chọn 1 từ (greedy decoding)",
     "Cải thiện chất lượng dịch 2-5% BLEU"),

    ("Subword (BPE)",
     "Chia từ thành đơn vị nhỏ hơn, xử lý từ hiếm tốt hơn (VD: 'playing' = 'play'+'ing')",
     "Giảm tỷ lệ <UNK> từ 5-10% xuống gần 0%"),

    ("Transformer",
     "Thay LSTM bằng self-attention, xử lý song song, nhanh và chính xác hơn",
     "SOTA cho dịch máy, BLEU tăng 10-15%"),
]

for i, (method, desc, impact) in enumerate(architecture_improvements, 1):
    print(f"\n{i}. {method}")
    print(f"   Mô tả: {desc}")
    print(f"   Tác động: {impact}")

print("\n\n CÁC KỸ THUẬT CẢI TIẾN HUẤN LUYỆN:")
print("=" * 60)

training_improvements = [
    ("ReduceLROnPlateau", "Tự động giảm learning rate khi val_loss không giảm"),
    ("Scheduled Sampling", "Giảm dần teacher forcing từ 1.0 → 0.5 theo epoch"),
    ("Data Augmentation", "Tăng dữ liệu bằng back-translation (dịch FR→EN→FR)"),
]

for i, (method, desc) in enumerate(training_improvements, 1):
    print(f"{i}. {method:25s} - {desc}")

print("\n" + "=" * 60)
print("HOÀN TẤT PHÂN TÍCH!")
print("=" * 60)